In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
table_19 = pd.read_excel("Table 19.xlsx")
clause_40_2_1_1 = pd.read_excel("clause 40_2_1_1.xlsx")
clause_26_2_1_1 = pd.read_excel('clause 26_2_1_1.xlsx')

In [3]:
axial_load = float (input("Enter axial load in kNm: "))
column_length = float (input("Enter length of column in mm:"))
column_breadth = float (input("Enter breadth of column  in mm:"))
diameter_bars = int (input("Enter the diameter of rod in mm [10, 12, 14, 16, 18, 20, 22, 25, 32, 40]: "))
num_bars = int (input("Enter the number of bars taken : "))
fck = int (input("Enter the grade of concrete from [20, 25, 30, 35, 40]: "))
fy = int (input("Enter the grade of steel from [415, 500]: "))
bearing_capacity_soil = float (input("Enter the bearing capacity of soil in kN/m*m: "))
if (column_breadth > column_length):
    column_length, column_breadth = column_breadth, column_length

Enter axial load in kNm: 1200
Enter length of column in mm:450
Enter breadth of column  in mm:450
Enter the diameter of rod in mm [10, 12, 14, 16, 18, 20, 22, 25, 32, 40]: 25
Enter the number of bars taken : 4
Enter the grade of concrete from [20, 25, 30, 35, 40]: 20
Enter the grade of steel from [415, 500]: 415
Enter the bearing capacity of soil in kN/m*m: 105


In [20]:
footing_length = np.sqrt(((1.1 * axial_load)/bearing_capacity_soil))
footing_breadth =  (column_breadth/column_length) *footing_length
soil_pressure = (1.5*axial_load)/(footing_length * footing_breadth)

M_u_x = soil_pressure * (((footing_length - (column_length * (10**(-3))))**2)/8)
M_u_y = soil_pressure * (((footing_breadth - (column_breadth * (10**(-3))))**2)/8)
if (fy == 415):
    Q = 0.36 * fck * 0.48 * (1 - 0.42*0.48)
else :
    Q = 0.36 * fck * 0.46 * (1 - 0.42*0.46)

depth = np.ceil((np.sqrt((M_u_x*1000)/Q)) / 10 ) * 10
diameter_footing = int (input("Enter the diameter of rod in mm [10, 12, 14, 16, 18, 20, 22, 25, 32, 40]: "))
effective_depth = depth - 75 -(diameter_footing/2)

Ast_x = (((0.5 *fck)/fy) * (1- np.sqrt(1-(4.598*( (M_u_x * (10**6)) /((footing_length * (10**3))*fck* (effective_depth**2))))))) * (footing_length * (10**3)) * effective_depth
Ast_y = (((0.5 *fck)/fy) * (1- np.sqrt(1-(4.598*( (M_u_y * (10**6)) /((footing_breadth * (10**3))*fck* (effective_depth**2))))))) * (footing_breadth * (10**3)) * effective_depth


#calculating k from clause 40_2_1_1
if (effective_depth <= 150):
    k=1.3
elif (effective_depth >=300):
    k = 1
else:
    k = 1.3 + ((effective_depth - 150)*(-0.3/150))


temp_shear_y = Ast_y/(footing_breadth * effective_depth)
if (( fck - 10)/5 <= 6):
    concrete_index_y = int(( fck - 10)/5)
else:
    concrete_index_y = 6
if (temp_shear_y<= 0.15):
    concrete_index_x = 1
    torque_c_y = .28
elif  (temp_shear_y>= 3):
    concrete_index_x = 13
    torque_c_y = .71
else:
    concrete_index_x = int((np.floor(temp_shear_y*100))//25) + (1)
    torque_c_y = (((temp_shear_y - table19.iloc[concrete_index_x,0])*(table19.iloc[concrete_index_x,concrete_index_y]-table19.iloc[concrete_index_x+ 1,concrete_index_y ]))/(table19.iloc[concrete_index_x,0]-table19.iloc[concrete_index_x+ 1,0 ])) + table19.iloc[concrete_index_x,concrete_index_y]
torque_prem = torque_c_y * k
v_u = soil_pressure * 1 * (((footing_length - (column_length/1000))/2) - (effective_depth/1000))
torque_vu = v_u / ( effective_depth)
print(f"footing_length {footing_length} column_length: {column_length}, efffective depth: {effective_depth}")
if (torque_prem < torque_vu):
    effective_depth = (((v_u) / (torque_prem)) + effective_depth)/2
    while (torque_vu > torque_prem):
        v_u = soil_pressure * 1 * (((footing_length - (column_length/1000))/2) - (effective_depth/1000))
        torque_vu = v_u / ( effective_depth)
        effective_depth += 50
#two way shear

ks = min(1, ((1/2)+ (column_breadth/column_length)))
torque_vp_prem = ks * 0.25 * np.sqrt(fck)

net_punching_force = (1.5* axial_load) - ((soil_pressure * (column_length + effective_depth) * ( column_breadth + effective_depth)) * (10**(-6)))
torque_vp = net_punching_force / (2*(column_length + effective_depth) * ( column_breadth + effective_depth) * effective_depth)
while (torque_vp_prem < torque_vp):
    effective_depth += 50 
    torque_vp = net_punching_force / (2*(column_length + effective_depth) * ( column_breadth + effective_depth) * effective_depth)

#steel reinforcement along long side
per_y = ((0.5 *fck)/fy) * (1- np.sqrt(1-(4.598*( (M_u_y * (10**6)) /((footing_breadth * (10**3))*fck* (effective_depth**2)))))) *100
Ast_y = max(0.12, per_y) * (1/100) * (footing_breadth * (10**3)) * effective_depth
spacing_y = min(footing_breadth * np.pi * (1/4) * (diameter_footing**2),300)
print(f"Provide {diameter_footing}mm bars at {spacing_y}mm")

#steel reinforcement along short side
per_x = ((0.5 *fck)/fy) * (1- np.sqrt(1-(4.598*( (M_u_x * (10**6)) /((footing_length * (10**3))*fck* (effective_depth**2)))))) *100
Ast_x = max(0.12, per_x * (1/100) * (footing_length * (10**3)) * effective_depth)
spacing_x = min(footing_length * np.pi * (1/4) * (diameter_footing **2), 300)
print(f"Provide {diameter_footing}mm bars at {spacing_x}mm")

#development length

development_length =  (diameter_footing * 0.87*fy)/(4*1.6*clause_26_2_1_1.iloc[0,int((fck/5)-3)])
#double bar
fbr = (1.5* axial_load*1000)/(column_length * column_breadth)
perm_bearing_stress = 0.45 * fck * min(2, max(1, np.sqrt((footing_length * footing_breadth)/(column_length * column_breadth))))
while (fbr > perm_bearing_stress):
    footing_length += 50
    footing_breadth += 50
    perm_bearing_stress = 0.45 * fck * min(2, max(1, np.sqrt((footing_length * footing_breadth)/(column_length * column_breadth))))
    
Ast_min = min(0.005 * ((column_length * column_breadth)),452)
num_bars = max(4, Ast_min/(np.pi * (1/4) * (diameter_footing **2)))
development_length_double_bar = (diameter_footing * 0.87*fy)/(4*1.6*clause_26_2_1_1.iloc[0,int((fck/5)-3)])/1.25

while(development_length_double_bar > effective_depth):
    development_length_double_bar = (diameter_footing * 0.87*fy)/(4*1.6*clause_26_2_1_1.iloc[0,int((fck/5)-3)])/1.25
    if (diameter_footing <=10):
        development_length_double_bar = effective_depth
        break
    diameter_footing -= 2
num_dowel_bars = Ast_min / (np.pi * (1/4) * (diameter_footing **2))        
print(f"dowel bars {num_dowel_bars}")
print(f"development length of footing{development_length}")
print(f"development length of double bar{development_length_double_bar}")


print(f"puch {net_punching_force}")
print(f"new effective depth {effective_depth}")
print(f"torque_c:{torque_c_y} torque_prem: {torque_prem}")
print(f"soil pressure{soil_pressure}")

print(f"v_u: {v_u}")
  
print(f"torque_vu = {torque_vu}")



Enter the diameter of rod in mm [10, 12, 14, 16, 18, 20, 22, 25, 32, 40]: 20
footing_length 3.5456210417116734 column_length: 450.0, efffective depth: 165.0
Provide 20mm bars at 300mm
Provide 20mm bars at 300mm
752.1874999999999
dowel bars 5.755042742202935
development length of footing940.2343749999999
development length of double bar292.2889123750059
puch 1721.107844876371
new effective depth 292.2889123750059
torque_c:0.71 torque_prem: 0.9017
soil pressure143.1818181818182
v_u: 186.92695757793715
torque_vu = 0.7715043818786822
